# Global Minimial Viable Products (1)
This is only for CONUS currently as the potential runoff product is to large to be an asset. 

Combine:
* SMAP available porosity
* ERA5-Merit potential runoff
* MERIT terrain slope
* MERIT upstream contributing area (needs to be less than some threshold)
* (ERA5 Moisture values)
* USDA soil texture class

In to various "products".

In [1]:
import ee
import numpy as np

import sys
sys.path.insert(1, '../datasets/SMAP-USDA/')  # until we package this
import visualize as vis
import water_common as common
import normalize as norm
from statistics import mean
import folium
from folium import plugins

Loading BokehJS ...

In [2]:
#ee.Authenticate()
ee.Initialize()

In [3]:
# Some common parameters
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
days_per_month = [
    31, 28, 31, 30, 31, 30,
    31, 31, 30, 31, 30, 31] 
seconds_per_month = [ dd* 24*60*60 for dd in days_per_month ]
month = 4
month_name = month_names[month - 1]

bbox = common.bboxes()['conus']

# SMAP-USDA Available Porosity
More available porosity is better.

In [4]:
# Get a single example image 
smap_usda_clim = ee.ImageCollection('users/jamesmcc/smap_usda_climatology')

avail_porosity_img = (
    smap_usda_clim
    .filter(ee.Filter.eq('month', month))
    .filter(ee.Filter.eq('band', 'avail_porosity_mm')).first())

avail_porosity_img_scaled = norm.img_scale(
    avail_porosity_img, area_of_interest=common.bboxes()['conus'])

In [5]:
# smap_range = norm.img_range(img, area_of_interest=common.bboxes()['conus'])
# smap_range

In [6]:
smap_missing = -99999

# units: mm
avail_porosity_data = np.array(
        avail_porosity_img
        .sampleRectangle(bbox, ['avail_porosity_mm'], smap_missing)
        .get('avail_porosity_mm')
        .getInfo())
avail_porosity_data_mask = np.ma.masked_values(avail_porosity_data, smap_missing)
avail_porosity_data_drop = avail_porosity_data_mask[~avail_porosity_data_mask.mask]

# Scaled
avail_porosity_scaled_data = np.array(
        avail_porosity_img_scaled
        .sampleRectangle(bbox, ['avail_porosity_mm'], smap_missing)
        .get('avail_porosity_mm')
        .getInfo())
avail_porosity_scaled_data_mask = np.ma.masked_values(avail_porosity_scaled_data, smap_missing)
avail_porosity_scaled_data_drop = avail_porosity_scaled_data_mask[~avail_porosity_scaled_data_mask.mask]

In [7]:
#print(avail_porosity_data)
#avail_porosity_data_mask
#avail_porosity_data_drop

In [8]:
_ = norm.normal_dist_plot(avail_porosity_data_drop)

In [9]:
_ = norm.normal_dist_plot(avail_porosity_scaled_data_drop)

## ERA5-MERIT Potential Runoff

In [10]:
# Get a single example image 
pot_runoff_clim = ee.ImageCollection(
    'users/jamesmcc/era5_sfc_runoff_conus_climatology')

pot_runoff_img = (
    pot_runoff_clim
    .filter(ee.Filter.eq('month', month)).first())

# Mask to a usable range
#runoff_mask = pot_runoff_img.lte(.5).And(pot_runoff_img.gte(.01))

pot_runoff_img_scaled = norm.img_scale(
    pot_runoff_img, 
    area_of_interest=common.bboxes()['conus'])

runoff_range = norm.img_range(pot_runoff_img_scaled, area_of_interest = bbox)
print(runoff_range)


[0, 1]


In [11]:
# Do the same for ERA5 moisture, get a single example image 
pot_moisture_clim = ee.ImageCollection(
    'users/amgadellaboudy/era5_vol_soil_moisture_conus_climatology1')

pot_moisture_img = (
    pot_moisture_clim
    .filter(ee.Filter.eq('month', month))
    .filter(ee.Filter.eq('variable', 'potential_sfc_moisture_mon_clim_cms')).first())

# No masking for moisture values
#moisture_mask = pot_moisture_img.lte(.5).And(pot_moisture_img.gte(.01))

pot_moisture_img_scaled = norm.img_scale(
    pot_moisture_img, 
    area_of_interest=common.bboxes()['conus'])


In [12]:
coos_county_parcels = ee.FeatureCollection(
    'users/tsongkapa/Coos_county_parcels')
douglas_county_parcels = ee.FeatureCollection(
    'users/tsongkapa/Douglas_county_parcels')

In [13]:
# Image.sampleRectangle: Too many pixels in sample; must be <= 262144. Got 2047606081.
# https://gis.stackexchange.com/questions/350771/earth-engine-simplest-way-to-move-from-ee-image-to-array-for-use-in-sklearn/351177#351177
# runoff_missing = -99999
# pot_runoff_data = np.array(
#         pot_runoff_img_scaled
#         .sampleRectangle(bbox, ['potential_sfc_runoff_mon_clim_cms'], smap_missing)
#         .get('potential_sfc_runoff_mon_clim_cms')
#         .getInfo())
# pot_runoff_data_mask = np.ma.masked_values(pot_runoff_data, smap_missing)
# pot_runoff_data_drop = pot_runoff_data_mask[~pot_runoff_data_mask.mask]

In [14]:
# print(pot_runoff_data)

In [15]:
# pot_runoff_data_mask
# pot_runoff_data_drop

In [16]:
# _ = norm.normal_dist_plot(avail_porosity_data_drop)

## MERIT Terrain Slope

In [17]:
#include slope as a factor in scoring product 1
slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope')
slope_mask = slope_img.lte(6)
slope_img_scaled = norm.img_scale(slope_img, area_of_interest=common.bboxes()['conus'])
# This dosent really need scaled, just a mask?

# Soil Types

In [18]:
#incorporate soil types as a factor in scoring land parcels
soil_types = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")

In [19]:
#categorizing soil types and depths based on retaining plant-available water, grouping top soils and bottom soils together
top_soils = [5,7,8,10]
medium_soils = [2,4,6,9]
low_soils = [1,3,11,12]
soil_0 = soil_types.expression(
        "(b('b0') == 12) ? 1.0" +
        ": (b('b0') == 11) ? 1.0" +
        ": (b('b0') == 10) ? 1.0" +
        ": (b('b0') == 9) ? 1.0" +
        ": (b('b0') == 8) ? 0.7" +
        ": (b('b0') == 7) ? 0.7" +
        ": (b('b0') == 6) ? 0.7" +
        ": (b('b0') == 5) ? 0.7" +
        ": (b('b0') == 4) ? 0.4" +
        ": (b('b0') == 3) ? 0.4" +
        ": (b('b0') == 2) ? 0.4" +
        ": (b('b0') == 1) ? 0.4" +
        ": 0")

soil_10 = soil_types.expression(
        "(b('b10') == 12) ? 1.0" +
        ": (b('b10') == 11) ? 1.0" +
        ": (b('b10') == 10) ? 1.0" +
        ": (b('b10') == 9) ? 1.0" +
        ": (b('b10') == 8) ? 0.7" +
        ": (b('b10') == 7) ? 0.7" +
        ": (b('b10') == 6) ? 0.7" +
        ": (b('b10') == 5) ? 0.7" +
        ": (b('b10') == 4) ? 0.4" +
        ": (b('b10') == 3) ? 0.4" +
        ": (b('b10') == 2) ? 0.4" +
        ": (b('b10') == 1) ? 0.4" +
        ": 0")

soil_30 = soil_types.expression(
        "(b('b30') == 12) ? 1.0" +
        ": (b('b30') == 11) ? 1.0" +
        ": (b('b30') == 10) ? 1.0" +
        ": (b('b30') == 9) ? 1.0" +
        ": (b('b30') == 8) ? 0.7" +
        ": (b('b30') == 7) ? 0.7" +
        ": (b('b30') == 6) ? 0.7" +
        ": (b('b30') == 5) ? 0.7" +
        ": (b('b30') == 4) ? 0.4" +
        ": (b('b30') == 3) ? 0.4" +
        ": (b('b30') == 2) ? 0.4" +
        ": (b('b30') == 1) ? 0.4" +
        ": 0")

soil_60 = soil_types.expression(
        "(b('b60') == 5) ? 1.0" +
        ": (b('b60') == 7) ? 1.0" +
        ": (b('b60') == 8) ? 1.0" +
        ": (b('b60') == 10) ? 1.0" +
        ": (b('b60') == 2) ? 0.7" +
        ": (b('b60') == 4) ? 0.7" +
        ": (b('b60') == 6) ? 0.7" +
        ": (b('b60') == 9) ? 0.7" +
        ": (b('b60') == 1) ? 0.3" +
        ": (b('b60') == 3) ? 0.3" +
        ": (b('b60') == 11) ? 0.3" +
        ": (b('b60') == 12) ? 0.3" +
        ": 0")

soil_100 = soil_types.expression(
        "(b('b100') == 5) ? 1.0" +
        ": (b('b100') == 7) ? 1.0" +
        ": (b('b100') == 8) ? 1.0" +
        ": (b('b100') == 10) ? 1.0" +
        ": (b('b100') == 2) ? 0.7" +
        ": (b('b100') == 4) ? 0.7" +
        ": (b('b100') == 6) ? 0.7" +
        ": (b('b100') == 9) ? 0.7" +
        ": (b('b100') == 1) ? 0.3" +
        ": (b('b100') == 3) ? 0.3" +
        ": (b('b100') == 11) ? 0.3" +
        ": (b('b100') == 12) ? 0.3" +
        ": 0")

soil_200 = soil_types.expression(
        "(b('b200') == 5) ? 1.0" +
        ": (b('b200') == 7) ? 1.0" +
        ": (b('b200') == 8) ? 1.0" +
        ": (b('b200') == 10) ? 1.0" +
        ": (b('b200') == 2) ? 0.7" +
        ": (b('b200') == 4) ? 0.7" +
        ": (b('b200') == 6) ? 0.7" +
        ": (b('b200') == 9) ? 0.7" +
        ": (b('b200') == 1) ? 0.3" +
        ": (b('b200') == 3) ? 0.3" +
        ": (b('b200') == 11) ? 0.3" +
        ": (b('b200') == 12) ? 0.3" +
        ": 0")

top_soils = soil_0.expression('top_soil + soil_10 + soil_30',
                             {'top_soil': soil_0.select('constant'),
                             'soil_10': soil_10.select('constant'),
                             'soil_30': soil_30.select('constant')})

bottom_soils = soil_60.expression('soil_60 + soil_100 + soil_200',
                                 {'soil_60': soil_60.select('constant'),
                                  'soil_100': soil_100.select('constant'),
                                  'soil_200': soil_200.select('constant')})

In [20]:
#scaling top soils and bottom soils
top_soils_scaled = norm.img_scale(top_soils, area_of_interest=common.bboxes()['conus'])
bottom_soils_scaled = norm.img_scale(bottom_soils, area_of_interest=common.bboxes()['conus'])

## Product 1
Simply give higher scores to places that have both high potential runoff and available porosity (and with low moisture), in only areas with acceptable slopes

In [21]:
product_1 = avail_porosity_img_scaled.multiply(pot_runoff_img_scaled).multiply(top_soils_scaled).multiply(bottom_soils_scaled).divide(slope_img_scaled).updateMask(slope_mask)

In [22]:
#understanding product_1 range

product_1_range = norm.img_range(product_1, area_of_interest = bbox)

palette_name = 'RdYlBu'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum=product_1_range[0], maximum= product_1_range[1])

box_corners = bbox.toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

vis_params = {
    'min': product_1_range[0], 'max': product_1_range[1], 'dimensions': 512,
    'palette': palette}



In [23]:
#sum up product_1 scores over douglas and coos county parcels to obtain a parcel score, obtain image, obtain image for bounding box we're interested in

parcel_score_douglas = product_1.reduceRegions(collection= douglas_county_parcels, reducer= ee.Reducer.max(), scale = 90)
parcel_score_douglas_2 = parcel_score_douglas.reduceToImage(properties = ['max'], reducer = ee.Reducer.firstNonNull())




In [24]:
parcel_score_coos = product_1.reduceRegions(collection= coos_county_parcels, reducer= ee.Reducer.max(), scale = 90)
parcel_score_coos_2 = parcel_score_coos.reduceToImage(properties = ['max'], reducer = ee.Reducer.firstNonNull())

bound_box = ee.Feature(ee.Geometry.Polygon([[[-124.1, 43.38], [-124.1, 42.88], [-123.6, 42.88], [-123.6, 43.38]]]))
bound_box_img = bound_box.getMapId()['image']

In [25]:
#calculate range of parcel score for douglas county
#parcel_score_douglas_range = norm.img_range(parcel_score_douglas_2, area_of_interest = bbox)

In [26]:
#creating the legend for parcel scores

palette_name = 'RdYlBu'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum =  0, maximum= 1)

vis_params_2 = {
    'min': 0, 'max': 1,'dimensions': 512,
    'palette': palette}


In [27]:
#creating the map, visualizing parcel score data for parcels in douglas and coos counties, option to visualize soil texture as well

the_map = vis.folium_map(location=[43.25, -123.5], zoom_start=8, height=500)
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_name = month_names[month-1]

palette_2 = ['black', 'yellow', 'brown', 'green']

douglas_img = douglas_county_parcels.getMapId()['image']

vis_params_3 = {
  'min': 0,
  'max': 1.0,
  'palette': palette}

vis_params_4 = {
  'min': 0,
  'max': 20.0,
  'palette': palette}

the_map.add_ee_layer(parcel_score_douglas_2, vis_params_2, name = 'Parcel Score_douglas')
the_map.add_ee_layer(parcel_score_coos_2, vis_params_2, name = 'Parcel Score_coos')
#the_map.add_ee_layer(top_soils_scaled, vis_params_3, name = 'Top Soils')
#the_map.add_ee_layer(bottom_soils_scaled, vis_params_3, name = 'Bottom Soils')
the_map.add_ee_layer(slope_img, vis_params_4, name = 'Slopes')
#the_map.add_ee_layer(pot_runoff_img_scaled, vis_params_3, name = 'Runoff')
#the_map.add_ee_layer(product_1, vis_params_3, name = 'Product_1')
#the_map.add_ee_layer(avail_porosity_img_scaled, vis_params_3, name = 'Porosity')
the_map.add_ee_layer(douglas_img, {}, name = 'Douglas_parcels')
the_map.add_ee_layer(bound_box_img, {}, name = 'Bounding_Box')
the_map.add_ee_layer(avail_porosity_img, vis_params_4, name = 'USDA Porosity')

vis.folium_display(the_map)


#douglas_img = douglas_county_parcels.draw(color = 'green', strokeWidth= 1)
#coos_img = coos_county_parcels.draw(color = 'green', strokeWidth= 1)

In [67]:
collection_name_d = (
    'users/amgadellaboudy/product_1_douglas')
product_1_d_asset = ee.data.createAsset(
    {'type': 'ImageCollection'}, collection_name_d)
description_d= 'Douglas_County_Water_Parcel_Score'

collection_name_c = (
    'users/amgadellaboudy/product_1_coos')
product_1_c_asset = ee.data.createAsset(
    {'type': 'ImageCollection'}, collection_name_c)
description_c= 'Coos_County_Water_Parcel_Score'
oregon_box = [[-124.1, 43.38], [-124.1, 42.88], [-123.6, 42.88], [-123.6, 43.38]]

xx_d = ee.batch.Export.image.toAsset(parcel_score_douglas_2, description = description_d, assetId = product_1_d_asset['id'] + '/' + description_d, region=oregon_box, scale = 90 )
xx_c = ee.batch.Export.image.toAsset(parcel_score_coos_2, description = description_c, assetId = product_1_c_asset['id'] + '/' + description_c, region= oregon_box, scale = 90)
xx_d.start()
xx_c.start()

## Product 2
Ratio of potential_runoff:available_porosity?

In [13]:
runoff_pixel_area_m2 = pot_runoff_img.pixelArea()
pot_runoff_img_mm = pot_runoff_img.divide(runoff_pixel_area_m2).multiply(seconds_per_month[month-1] * 1000)
# pot_runoff_mask = pot_runoff_img_mm.gte(1000)
avail_porosity_mask = avail_porosity_img.gte(50)

In [14]:
norm.img_range(pot_runoff_img_mm.updateMask(runoff_mask), area_of_interest=bbox)
#norm.img_range(pot_runoff_img_mm, area_of_interest=bbox)

[16.847965699571173, 1162.8066667861892]

In [15]:
norm.img_range(runoff_pixel_area_m2, area_of_interest=bbox)

[1147915.625, 1592437.625]

In [16]:
norm.img_range(avail_porosity_img, area_of_interest=bbox)

[0, 205.34410095214844]

In [17]:
product_2_raw = (
    pot_runoff_img_mm
    .divide(avail_porosity_img)
    .updateMask(slope_mask)
    .updateMask(avail_porosity_mask)
    .updateMask(runoff_mask))
product_2_mask = product_2_raw.gte(1)
product_2 = product_2_raw.updateMask(product_2_mask)

In [18]:
product_2_range = norm.img_range(product_2, area_of_interest=bbox)


palette_name = 'BrBG'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum=product_2_range[0], maximum=product_2_range[1])

box_corners = bbox.toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

vis_params = {
    'min': product_2_range[0], 'max': product_2_range[1], 'dimensions': 512,
    'palette': palette}

In [19]:
the_map = vis.folium_map(location=[center_lat, center_lon], zoom_start=4, height=500)
the_map.add_ee_layer(product_2, vis_params, 'Product 2: ' + month_name)
vis.folium_display(the_map)